# **Efficient Yelp API calls**
## *Jaco van Wyk*
-----------------------------

In [34]:
# Imports

# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook


In [35]:
# Loading Yelp API keys

import json
with open('/Users/javwyk/.secret/yelp_api.json') as f:
    login = json.load(f)
login.keys()


dict_keys(['client-id', 'api-key'])

In [36]:
# Create an instance with your key
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api


In [37]:
# set our API call parameters 
LOCATION = 'San Diego, CA'
TERM = 'Seafood'


In [38]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_SD_Seafood.json"
JSON_FILE


'Data/results_in_progress_SD_Seafood.json'

In [46]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  


In [47]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages


[!] Data/results_in_progress_SD_Seafood.json already exists. Deleting previous file...
[i] Data/results_in_progress_SD_Seafood.json not found. Saving empty list to new file.
- 0 previous results found.


140

In [48]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)


  0%|          | 0/140 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [49]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,mbG2JAAQ33kKuO3PMPUObg,ironside-fish-and-oyster-san-diego,Ironside Fish & Oyster,https://s3-media2.fl.yelpcdn.com/bphoto/vrwLSt...,False,https://www.yelp.com/biz/ironside-fish-and-oys...,5276,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.5,"{'latitude': 32.72273395085638, 'longitude': -...",[delivery],$$,"{'address1': '1654 India St', 'address2': '', ...",+16192693033,(619) 269-3033,7663.918882
1,Sa_qcnc7ZgzSOylf3plTRA,c-level-san-diego-6,C Level,https://s3-media4.fl.yelpcdn.com/bphoto/rE2Ti2...,False,https://www.yelp.com/biz/c-level-san-diego-6?a...,4467,"[{'alias': 'newamerican', 'title': 'New Americ...",4.5,"{'latitude': 32.724137, 'longitude': -117.188594}",[],$$,"{'address1': '880 Harbor Island Dr', 'address2...",+16192986802,(619) 298-6802,8065.690253
2,ccLW5mhIAC_GIbuYfj10JQ,boxing-crab-san-diego,Boxing Crab,https://s3-media2.fl.yelpcdn.com/bphoto/pRFY-r...,False,https://www.yelp.com/biz/boxing-crab-san-diego...,113,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",5.0,"{'latitude': 32.76754, 'longitude': -117.0552}","[delivery, pickup]",NaN,"{'address1': '6663 El Cajon Blvd', 'address2':...",+16194677290,(619) 467-7290,9612.097987
3,PUcvlST3Q9b94nKMkS5wUg,blue-water-san-diego-2,Blue Water,https://s3-media3.fl.yelpcdn.com/bphoto/_ThW3-...,False,https://www.yelp.com/biz/blue-water-san-diego-...,4170,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",4.0,"{'latitude': 32.7419837556966, 'longitude': -1...",[delivery],$$,"{'address1': '3667 India St', 'address2': None...",+16194970914,(619) 497-0914,5918.068252
4,x8QP2bvziC4dvtz17PtV_A,shrimp-heads-san-diego-2,Shrimp Heads,https://s3-media3.fl.yelpcdn.com/bphoto/tqSE4J...,False,https://www.yelp.com/biz/shrimp-heads-san-dieg...,1271,"[{'alias': 'cajun', 'title': 'Cajun/Creole'}, ...",4.5,"{'latitude': 32.75561, 'longitude': -117.133084}","[delivery, pickup]",$$,"{'address1': '2832 El Cajon Blvd', 'address2':...",+16197388184,(619) 738-8184,4352.332728


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,-O-9y0Ax0WKZniG6t8VTQw,fifty9fifty-local-eatery-san-diego,Fifty9Fifty Local Eatery,https://s3-media3.fl.yelpcdn.com/bphoto/UG06Wg...,False,https://www.yelp.com/biz/fifty9fifty-local-eat...,136,"[{'alias': 'tradamerican', 'title': 'American'...",3.5,"{'latitude': 32.839366, 'longitude': -117.098668}","[pickup, delivery]",$$,"{'address1': '5950 Santo Rd', 'address2': '', ...",+18587374665,(858) 737-4665,7500.576767
996,-joWy7rbpYCLihBEUVa47g,ramen-ryoma-san-diego-3,Ramen Ryoma,https://s3-media3.fl.yelpcdn.com/bphoto/rnlNps...,False,https://www.yelp.com/biz/ramen-ryoma-san-diego...,231,"[{'alias': 'ramen', 'title': 'Ramen'}, {'alias...",4.0,"{'latitude': 32.7482060990798, 'longitude': -1...","[pickup, delivery]",$$,"{'address1': '815 University Ave', 'address2':...",+16195397296,(619) 539-7296,4717.404430
997,wELhihJnj8P-CMAJsplSOQ,mariscos-negro-durazo-de-culiacán-sinaloa-tijuana,Mariscos Negro Durazo de Culiacán Sinaloa,https://s3-media1.fl.yelpcdn.com/bphoto/gbwzi6...,False,https://www.yelp.com/biz/mariscos-negro-durazo...,19,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",3.5,"{'latitude': 32.47804, 'longitude': -116.96901}",[],$$,{'address1': 'Blvd. Gral. Rodolfo Sánchez Tabo...,+526646224212,+52 664 622 4212,38833.178995
998,66ZXt0fExG4RF7cvxNmdcw,carin-de-ria-encinitas-5,CARiN de RiA,https://s3-media3.fl.yelpcdn.com/bphoto/NVVzoz...,False,https://www.yelp.com/biz/carin-de-ria-encinita...,446,"[{'alias': 'filipino', 'title': 'Filipino'}, {...",4.5,"{'latitude': 33.039585, 'longitude': -117.293675}","[pickup, delivery]",$$,"{'address1': '124 West I St', 'address2': '', ...",+17605574873,(760) 557-4873,30601.977980
999,gMf2OeqEeCZneDlgLL8BIw,cottage-drive-in-liquor-san-diego,Cottage Drive-In Liquor,https://s3-media1.fl.yelpcdn.com/bphoto/7gRSrx...,False,https://www.yelp.com/biz/cottage-drive-in-liqu...,37,"[{'alias': 'beer_and_wine', 'title': 'Beer, Wi...",4.0,"{'latitude': 32.74589, 'longitude': -117.14647}","[pickup, delivery]",$$,"{'address1': '3747 Park Blvd', 'address2': '',...",+16192954391,(619) 295-4391,4998.903717


In [50]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()


0